# Imports

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import random
import time
import os
import sys
import copy
import math
import pickle
import json
import pandas as pd
import dataset
import transformers
import trl
import tqdm
from collections import namedtuple
from itertools import count
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from collections import deque
from collections import namedtuple
from itertools import count
from itertools import product
from collections import OrderedDict

In [ ]:
# 14 things
# Spend 15 mins on things = 3.5 hours

# Debugging and pretty-printing

TODO:
- print in color
- link to instrs for VSCode debugger
- how to debug in JNotebook
- memory stuff (debugging, printing)
- printing stack trace in error message
- timing
- color in JNotebook

In [ ]:
def printc(text, color):
    """
    Prints the given text in the specified color.

    :param text: The text to be printed
    :param color: The color in which the text is to be printed. 
                  Accepts 'red', 'green', 'yellow', 'blue', 'magenta', 'cyan', 'white'.
    """
    colors = {
        "red": "\033[91m",
        "green": "\033[92m",
        "yellow": "\033[93m",
        "blue": "\033[94m",
        "magenta": "\033[95m",
        "cyan": "\033[96m",
        "white": "\033[97m",
    }

    # Check if the specified color is valid
    if color not in colors:
        print("Invalid color. Choose from 'red', 'green', 'yellow', 'blue', 'magenta', 'cyan', 'white'.")
        return

    # Print the text in the specified color
    print(f"{colors[color]}{text}\033[0m")

# Numpy stuff

TODO:
- remind myself what broadcasts
- fancy indexing
- select

# Pandas stuff

TODO:
- Printing dataset stats
- deduplicating
- merging
- groupby
- pivot
- loading

# Data loading and datasets

# Data visualization

# TODO:
- DIFFERENT SETTING
  - imgs
  - diff

  

In [ ]:
df = pd.read_csv('data.csv')
vals = df['vals']
# Pandas can compute all the standard stats at once


In [ ]:
# Data stuff
import numpy as np
import pandas as pd

def analyze_data(x, y):
    """
    Analyzes the given data and prints the following:
    - The shape of the data
    - The number of unique values in the data
    - The mean of the data
    - The standard deviation of the data
    - The minimum value in the data
    - The maximum value in the data

    :param x: The input data
    :param y: The output data
    """
    print(f"Shape of x: {x.shape}")
    print(f"Shape of y: {y.shape}")
    print(f"Number of unique values in x: {len(np.unique(x))}")
    print(f"Number of unique values in y: {len(np.unique(y))}")
    print(f"Mean of x: {np.mean(x)}")
    print(f"Mean of y: {np.mean(y)}")
    print(f"Standard deviation of x: {np.std(x)}")
    print(f"Standard deviation of y: {np.std(y)}")
    print(f"Minimum value in x: {np.min(x)}")
    print(f"Minimum value in y: {np.min(y)}")
    print(f"Maximum value in x: {np.max(x)}")
    print(f"Maximum value in y: {np.max(y)}")
    print(f"number of NaNs in x: {np.sum(np.isnan(x))}")
    print(f"number of NaNs in y: {np.sum(np.isnan(y))}")
    print(f"number of infs in x: {np.sum(np.isinf(x))}")
    print(f"number of infs in y: {np.sum(np.isinf(y))}")
    
    # Print dtypes
    # Check duplicates
    # check dataset balance
    # check linear regression acc
    
    # For images, print representative samples
    # For text, print representative samples
    # For text, print lengths
    
    
    
    

# Data cleaning

# PyTorch stuff

TODO:
- Training loop
- contiguous, etc.
- weirdness (e.g. max)
- examples of common networks
- gotchas
- debugging (e.g. by printing param values)
- common hyperparameters
- LR schedule
- common initializations
- NLL??
- gradient clipping

In [ ]:
# PyTorch training loop

batch_size = 32
lr = 1e-3  # For training from scratch; for finetuning, use 1e-4
weight_decay = ... # TODO: figure this out!

model = torch.Linear(3, 3)
# Getting a pretrained model:
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
# Transformer from scratch
# model = nn.TransformerDecoderLayer(
#     d_model, 
#     nhead, 
#     dim_feedforward=2048, 
#     dropout=0.1, 
#     activation=<function relu>, 
#     layer_norm_eps=1e-05, 
#     batch_first=False, 
#     norm_first=False, 
#     bias=True, 
#     device=None, 
#     dtype=None
# )



optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)  # Either use Adam or AdamW
# adamw_optimizer = optim.AdamW(model.parameters(), lr=lr)

# This takes in logits, not probabilities
# Shape of logits is [B, V, ...] where B is batch size and V is the number of classes (NOT B, S, V)
# Targets is [B, ...] where B is batch size and contains int idxs
loss_fn = nn.CrossEntropyLoss(
    # ignore_index=-100,  # -100 by default; set this to handle padding
    # reduction='mean',  # 'mean' by default; set this to 'none' if you want to get the individual losses; all the loss fns have this option
    # label_smoothing=0,  # I never use this, but apparently it's still a thing
) 
# loss_fn = nn.MSELoss()  # For regression tasks
# loss_fn = nn.BCEWithLogitsLoss()  # For binary classification tasks; takes in logits. If you have probs, use nn.BCELoss

lr_scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)


# Cuda stuff
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)



# Dummy dataset
inputs = torch.rand(100, 3)
targets = torch.randint(0, 3, (100,))
train_dataset = torch.utils.data.TensorDataset(inputs, targets)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(inputs, targets)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


def eval()


def train_loop(dataloader, model, loss_fn, optimizer, val_dataloader=None, epochs=10):
    size = len(dataloader.dataset)
    
    # Expects a dataloader
    for i, X in enumerate(dataloader):
        X = X.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, X)  # Change X to y if you have labels
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        if i % 100 == 0:
            loss, current = loss.item(), i * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return loss




# Sklearn stuff

# TRL stuff

# LLM stuff

# Metrics/plotting

# Multiprocessing

# Errors

# Git